In [1]:
from datetime import datetime
import time
import os
import sys

In [2]:
atlas_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/x64/Release"
sys.path.append(atlas_path)
from AtlasPy.core import Hydra, Portfolio, Strategy
from AtlasPy.ast import *

In [3]:
exchange_id = "test"
portfolio_id = "test_p"
strategy_id = "test_s"
exchange_path_fast = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/AtlasPy/test/data_fast.h5"
initial_cash = 100.0

ret_fast = 0.6351860771192923

hydra = Hydra()

In [4]:
exchange = hydra.addExchange(exchange_id, exchange_path_fast)
portfolio = hydra.addPortfolio(portfolio_id, exchange, initial_cash)
hydra.build()

In [5]:
read_fast = AssetReadNodeWrapper.make("ma_fast", 0, exchange)
read_slow = AssetReadNodeWrapper.make("ma_slow", 0, exchange)
spread = AssetDifferenceNodeWrapper.make(read_fast, read_slow)
op_variant = AssetOpNodeVariant.make(spread)

filter = ExchangeViewFilter(ExchangeViewFilterType.GREATER_THAN, 0.0)
exchange_view = ExchangeViewNodeWrapper.make(exchange, op_variant, filter)
allocation = AllocationNodeWrapper.make(exchange_view)
strategy_node = StrategyNodeWrapper.make(allocation, portfolio)
strategy = hydra.addStrategy(Strategy(strategy_id, strategy_node, 1.0))

In [6]:
time_sum = 0
n = 10
for i in range(n):
    st = time.perf_counter_ns()
    hydra.run()
    et = time.perf_counter_ns()
    time_sum += et - st

avg_time_micros = (time_sum / n) / 1000

tr = (strategy.getNLV() - initial_cash) / initial_cash
print(f"Time elapsed Avg: {avg_time_micros:.3f} us")
print(f"Total return: {tr:.3%}")
print(f"Epsilon: {tr - ret_fast}")

Time elapsed Avg: 21.780 us
Total return: 63.519%
Epsilon: 1.7763568394002505e-15


In [7]:
vbt_time_milis = 18.029
vbt_time_micros = vbt_time_milis * 1000
speedup = vbt_time_micros / avg_time_micros
print(f"Speedup: {speedup:.3f}x")

Speedup: 827.778x


In [8]:
(et - st)

18500